In [69]:


from google.cloud import bigquery
from google.cloud import secretmanager

import os
import pandas as pd
import db_dtypes
from dotenv import load_dotenv


def fetch_data_by_date(pub_date=None, table='table_1', project_id='tomastestproject-433206', dataset='testdb_1'):
    # load_dotenv()
    # Ange sökvägen till din service account JSON-fil
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'tomastestproject-433206-adc5bc090976.json'

    table_id = f"{project_id}.{dataset}.{table}"
    # Skapa en BigQuery-klient
    client = bigquery.Client()

    query = f"""
        SELECT company,title,description,publishedAt
        FROM `{table_id}`
        WHERE LEFT(publishedAt, 10) = '{pub_date}';

        """

    query_job = client.query(query)

    # Hämta resultaten
    results = query_job.result()

    return results.to_dataframe()

In [40]:
# from get_data import fetch_data_by_date
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
from datetime import datetime, timedelta

nltk.download([
    "vader_lexicon"
])


def make_score(string):
    sia = SentimentIntensityAnalyzer()
    if string is None:
        return None
    else:
        return sia.polarity_scores(string)['compound']

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/tomasrydenstam/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [7]:


from google.cloud import bigquery
from google.cloud import secretmanager

import os
import pandas as pd
import db_dtypes
from dotenv import load_dotenv


def fetch_data_by_date(pub_date=None, table='table_1', project_id='tomastestproject-433206', dataset='testdb_1'):
    # load_dotenv()
    # Ange sökvägen till din service account JSON-fil
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'tomastestproject-433206-adc5bc090976.json'

    table_id = f"{project_id}.{dataset}.{table}"
    # Skapa en BigQuery-klient
    client = bigquery.Client()

    query = f"""
        SELECT company,title,description,publishedAt
        FROM `{table_id}`
        WHERE LEFT(publishedAt, 10) = '{pub_date}';

        """

    query_job = client.query(query)

    # Hämta resultaten
    results = query_job.result()

    return results.to_dataframe()


# Exempel på användning av funktionen
if __name__ == "__main__":
    fetch_data_by_date("2024-08-20")
    # secret = access_secret_version(
    #     name='projects/839243415895/secrets/test-github-oauthtoken-e3deeb')
    # print(f"Fetched secret: {secret}")

/Users/tomasrydenstam/Desktop/Skola/test_project/testv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [70]:
date = '2024-08-20'
data = fetch_data_by_date('2024-08-20')

/Users/tomasrydenstam/Desktop/Skola/test_project/testv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [105]:
df_copy = data.copy()

In [106]:
df_copy

,company,title,description,publishedAt
0,JPM,Podcast: JPM connects to French payments netwo...,J.P. Morgan is joining French payments network...,2024-08-20T22:41:34Z
1,APPL,Best Lightweight Laptop for 2024,Looking for an ultralight ultraportable? These...,2024-08-20T16:55:00Z
2,APPL,Best Lightweight Laptop for 2024,Looking for an ultralight ultraportable? These...,2024-08-20T16:55:00Z
3,APPL,Control Center has been supercharged in iOS 18...,"In iOS 18, Apple has updated Control Center wi...",2024-08-20T10:12:27Z
4,APPL,Control Center has been supercharged in iOS 18...,"In iOS 18, Apple has updated Control Center wi...",2024-08-20T10:12:27Z
...,...,...,...,...
100,NVDA,Jim Cramer on NVIDIA (NVDA): ‘People Feel Ther...,None,2024-08-20T13:16:49Z
101,APPL,Apple Announces New NFC In-App Transaction Cap...,Apple says developers have new NFC in-app tran...,2024-08-20T15:43:21Z
102,APPL,Apple Announces New NFC In-App Transaction Cap...,Apple says developers have new NFC in-app tran...,2024-08-20T15:43:21Z
103,APPL,iPhone 16 launch date leaked!,"According to the latest rumors, here is when w...",2024-08-20T10:17:30Z


In [174]:
df_copy['score_description'] = df_copy['description'].apply(make_score)
df_copy['score_title'] = df_copy['title'].apply(make_score)
means = df_copy.groupby(
    'company',)[['score_title', 'score_description']].mean()
means.reset_index(inplace=True)
means['pub_date'] = datetime.now().strftime(date)
means['fetch_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [180]:

def write(data,  table='news_sentiment', project_id='tomastestproject-433206', dataset='testdb_1') -> None:

    # Initiera BigQuery-klienten
    client = bigquery.Client.from_service_account_json(
        '/Users/tomasrydenstam/Downloads/tomastestproject-433206-adc5bc090976.json')

    # Definiera din dataset och tabell
    table_id = f"{project_id}.{dataset}.{table}"
    # print(table_id)
    list_of_dicts = data.to_dict(orient='records')
    # print(rows_to_insert)
    client.get_table(table_id)
    errors = client.insert_rows_json(table_id, list_of_dicts)
    print(errors)

In [175]:
means

,company,score_title,score_description,pub_date,fetch_date
0,APPL,0.044651,0.181734,2024-08-20,2024-08-26 14:04:13
1,JPM,-0.073580,0.052360,2024-08-20,2024-08-26 14:04:13
2,NVDA,0.002783,0.175400,2024-08-20,2024-08-26 14:04:13


In [158]:
list_of_dicts

[{'company': 'APPL',
  'score_title': 0.04465106382978724,
  'score_description': 0.18173404255319148,
  'pub_date': Timestamp('2024-08-20 00:00:00'),
  'fetch_date': Timestamp('2024-08-26 13:28:16.020288')},
 {'company': 'JPM',
  'score_title': -0.07358,
  'score_description': 0.052360000000000004,
  'pub_date': Timestamp('2024-08-20 00:00:00'),
  'fetch_date': Timestamp('2024-08-26 13:28:16.020288')},
 {'company': 'NVDA',
  'score_title': 0.002783333333333341,
  'score_description': 0.1754,
  'pub_date': Timestamp('2024-08-20 00:00:00'),
  'fetch_date': Timestamp('2024-08-26 13:28:16.020288')}]

In [147]:
means

,company,score_title,score_description,pub_date,fetch_date
0,APPL,0.044651,0.181734,2024-08-20,2024-08-26 13:28:16.020288
1,JPM,-0.073580,0.052360,2024-08-20,2024-08-26 13:28:16.020288
2,NVDA,0.002783,0.175400,2024-08-20,2024-08-26 13:28:16.020288


In [154]:
ls = means.drop(columns=['pub_date', 'fetch_date'])

In [170]:
ls

,company,score_title,score_description
0,APPL,0.044651,0.181734
1,JPM,-0.073580,0.052360
2,NVDA,0.002783,0.175400


In [152]:
means

0    APPL
1     JPM
2    NVDA
Name: company, dtype: object

In [182]:
write(means)

[]
